# Анализ рынка заведений общественного питания Москвы  
**Описание проекта:**  

Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.  

Для начала они просят вас — аналитика — подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.  
 
Вам доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер. 

**Описание данных:**  
Данные с информацией о заведениях хранятся в файле *moscow_places.csv.*  

**Цели исследования:**  
- Подготовить исследование рынка заведений общественного питания Москвы  
- Найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места  

**Ход исследования:**  

Проект будет состоять из пяти этапов:  
1. Обзор данных  
2. Предобработка данных 
3. Анализ данных  
4. Общий вывод по исследуемым данным
5. Детализация исследования: открытие кофейни

## Обзор данных

In [ ]:
# импорт библиотек
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import plotly.express as px
from matplotlib import pyplot as plt
import re

In [ ]:
# прочитаем csv-файл и сохраним данные в переменную
try:
    data = pd.read_csv('./moscow_places.csv')
except:
    data = pd.read_csv('/datasets/moscow_places.csv')

In [ ]:
# выведем первые 5 строк датафрейма data
data.head()

In [ ]:
# Выведем основную информацию о датафрейм с помощью метода info():
data.info()

In [ ]:
# посчитаем, сколько всего заведений представлено в данных:
print('Количество заведений общественного питания: ', data['name'].count())

In [ ]:
# посмотрим, сколько заведений представлено в каждой категории:
data['category'].value_counts()

Исходные данные содержат следующую информацию:  

*name* — название заведения    
*address* — адрес заведения    
*category* — категория заведения   
*hours* — информация о днях и часах работы  
*lat* — широта географической точки, в которой находится заведение  
*lng* — долгота географической точки, в которой находится заведение    
*rating* — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0)    
*price* — категория цен в заведении, например «средние», «ниже среднего», «выше среднего»  
*avg_bill* — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:  
- «Средний счёт: 1000–1500 ₽»  
- «Цена чашки капучино: 130–220 ₽»  
- «Цена бокала пива: 400–600 ₽» и так далее   

*middle_avg_bill* — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:  
- Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений  
- Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число  
- Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт  

*middle_coffee_cup* — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:  
- Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений  
- Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число  
- Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт  

*chain* — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):  
- 0 — заведение не является сетевым  
- 1 — заведение является сетевым  

*district* — административный район, в котором находится заведение, например Центральный административный округ  
*seats* — количество посадочных мест  

На первом этапе была выявлена следующие проблемы в исходных данных:  
- отсутствие единого формата записи наименований заведений(разный регистр, использование символов, английских букв)  
- большое количество пропусков в данных   

В рамках предобработки постараемся по-возможности устранить эти проблемы, чтобы подготовить данные к дальнейшему исследованию.

## Предобработка данных <a id="preprocessing"></a>

### Работа с дубликатами
####  Проверим данные на наличие явных дубликатов:

In [ ]:
print('Количество явных дубликатов:', data.duplicated().sum())

Явных дубликатов в исходных данных не обнаружено.

#### Перейдем к поиску неявных дубликатов    

##### Столбец name

In [ ]:
# сохраним уникальные наименования заведений в переменную names и выведем их на экран
names = sorted(data['name'].unique())
names

Беглого вгляда на названия заведений достаточно, чтобы заметить, что некоторые наименования заведений записаны по-разному, например,  '9 bar coffe' и '9 bar coffee'. Такие случаи указывают скорее на наличие опечаток и ошибочную запись. Ошибочная запись наименований заведений может повлечь за собой неточности при опрелении того, является ли заведением сетевым или нет.   

Попробуем устранить разности наименований в случаях, похожих на опечатки. Для этого:
- приведем наименования к нижнему регистру     
- исключим из наименований букву ё и специальные символы (все знаки, не являющиеся буквой или цифрой)

In [ ]:
# сохраним копию датафрейма с исходными названиями 
data_old = data.copy()

In [ ]:
# приведем названия заведений к нижнему регистру и заменим букву ё на е
data['name'] = data['name'].apply(lambda x: str.lower(x).replace('ё', 'е'))

In [ ]:
# сохраним уникальные названия заведений в переменную unique_names
unique_names = ",".join(data['name'].dropna().unique())
unique_names[:550]

In [ ]:
# создадим переменную symb и посчитаем как часто встречается каждый из символов
symb = re.split('\w+', unique_names, flags=re.IGNORECASE)
pd.Series(symb).value_counts()

Большинство символов не несут никакой смысловой назгрузки.  
Предлагаю убрать большую часть символов, оставив только те, убрав которые наименование потеряет различительную способость.

In [ ]:
# напишем регулярное выражение, которое заменит специальные символы (кроме / №) на пустую строку
# и применим его к столбцу с названиями заведений
data = data[data['name'].apply(lambda x: re.sub(r'[^\w/№]+', ' ', x) == x)]
data.head()

Попробуем вручную исключить самые очевидные опечатки:

In [ ]:
data['name'] = data['name'].replace({
    '9 bar coffe': '9 bar coffee',
    '9 bar coffeee': '9 bar coffee',
    'cafe13': 'cafe 13',
    'cofe fest': 'cofefest',
    'drive cafe': 'drive café',
    'main food': 'mainfood',
    'meat лав': 'meatлав',
    'shake up': 'shakeup',
    'pho bo': 'phobo',
    'барокат': 'баракат',
    'донер24': 'донер 24',
    'намангале': 'на мангале',
    'кружкапаб': 'кружка паб',
    'стардог': 'стардогс',
    'ситипицца': 'сити пицца',
    'сушисет': 'суши сет',
    'сушистор': 'суши стор',
    'чай хана халал': 'чайхана халал',
    'чайхана24': 'чайхана 24',
    'чайхона бишкек сити': 'чайхана бишкек сити',
    'чайхона ош сити': 'чайхана ош сити',
    'чайхана халал': 'чайхана халаль',
    'чайхона халва': 'чайхана халва',
    'чайхона №1': 'чайхона № 1',
    'шаурма24': 'шаурма 24'}, regex=True)

##### Столбец category

In [ ]:
# сгруппируем заведения по названию и посчитаем количество категорий для каждого из них
cat = data.groupby(['name', 'chain']).agg(
    {'category': lambda x: len(set(x))}).sort_values('category', ascending=False).reset_index()

cat

In [ ]:
# исключим заведения, которые после группировки имеют одну категорию
cat = cat[cat['category'] != 1]

In [ ]:
# рассмотрим некоторые примеры заведений
# пример 1
data[data['name'] == 'pho city']

In [ ]:
# пример 2
data[data['name'] == 'кондитерская олега ильина']

In [ ]:
# пример 3
data[data['name'] == 'натахтари'] 

Как мы видим в примерах, у многих заведений, являющихся одной сетью, разнятся категории.  
Предлагаю привести сети с одним и тем же названием к наиболее всречаемой категории среди указанных для этой сети.

In [ ]:
# используем класс counter() из модуля collections для подсчета частоты категорий 
from collections import Counter

# сохраним все сетевые заведения в переменную chains
chains = data[data['chain'] == 1]

In [ ]:
# сгруппируем заведения по названию и определим для каждого наиболее часто встречаемую категорию
grouped = chains.groupby('name', as_index=False).agg({'category': lambda x: Counter(x).most_common()[0][0]})
grouped.head()

In [ ]:
# создадим словарь с названиями заведений и их категориями 
replacing_cat = dict(zip(grouped['name'], grouped['category']))

# заменим категории в датафрейме в случае, если заведение встречается в словаре
new_cat = data[data['name'].isin(replacing_cat)]['name'].map(replacing_cat).to_list()
data.loc[data['name'].isin(replacing_cat), 'category'] = new_cat

##### Проверим, есть ли в данных заведения с одинаковым названием, категорией, районом и адресом:

In [ ]:
# сохраним результат проверки в переменную bool_series
bool_series = data.duplicated(subset=['name', 'category', 'district', 'address'])
 
# отобразим данные
data[bool_series]

Выявлено 4 заведения с совпадающими названием, категорией, районом и адресом.  
Рассмотим каждое из них:

In [ ]:
# заведение 1
data[data['name'] == 'more poke']

In [ ]:
# заведение 2
data[data['name'] == 'раковарня клешни и хвосты']

In [ ]:
# заведение 3
data[data['name'] == 'хлеб да выпечка']

In [ ]:
# заведение 4
data[data['name'] == 'cafe 13']

Дубликаты действительно есть. Удалим их при помощи метода *drop_duplicates():*

In [ ]:
data = data.drop_duplicates(subset=['name', 'category', 'district', 'address'])

Подведем итог и посчитаем, какое количество данных было почищено после предобработки столбцов *name* и *category*:

In [ ]:
print('Почищено уникальных наименований: {:.2%}'.format(
    1 - (data['name'].nunique() / data_old['name'].nunique())))

In [ ]:
print('Почищено категорий: {:.2%}'.format(
    1 - (data['category'].count() / data_old['category'].count())))

In [ ]:
print('Количество заведений после предобработки:', data['name'].count())

Работа с поиском неявных дубликатов завершена.

### Работа с аномалиями и выбросами  

Проверим распределение значений по среднему чеку и стоимости чашки кофе.   
Построим их графики, исключив NaN-значения и нули:

In [ ]:
# средний чек
sns.histplot(
    data[(~data['middle_avg_bill'].isna()) & (data['middle_avg_bill'] != 0)], 
    x='middle_avg_bill', bins=30, log_scale=True).set(
    title= f'Средний чек',
    xlabel='Размер счета, руб.',
    ylabel='Количество заведений')
plt.show()

Средний чек в заведениях составляет около 1 000 рублей. При этом есть заведения с аномально большими чеками ( более 10 000 рублей). Посмотрим медианные значения и квартили столбца с помощью метода describe():

In [ ]:
# исключим NaN и нули
bill = data[(~data['middle_avg_bill'].isna()) & (data['middle_avg_bill'] != 0)]

bill['middle_avg_bill'].describe()

Большая часть значений лежит в диапазоне до 1250 рублей.   
Для дальнейшего анализа предлагаю исключить аномально большие чеки и ограничить выборку заведениями с чеком **до 5 000 рублей.**   
Значения выше максимального примем за аномальные и исключим из выборки, заменив их на NaN:

In [ ]:
# замена аномалий на NaN
data.loc[data['middle_avg_bill'] > 5000, 'middle_avg_bill'] = np.nan 

Посмотрим на распределение стоимости чашки кофе:

In [ ]:
# средняя стоимость чашки кофе

sns.histplot(
    data[(~data['middle_coffee_cup'].isna()) & (data['middle_coffee_cup'] != 0)], 
    x='middle_coffee_cup', bins=30, log_scale=True).set(
    title= f'Средняя стоимость чашки кофе',
    xlabel='Цена за чашку кофе',
    ylabel='Количество заведений')
plt.show()

Cтоимость чашки кофе в большей части заведений составляет 150 - 200 рублей. Аномально большие значения единичны, рассмотрим их поближе:

In [ ]:
data[data['middle_coffee_cup'] > 1000]

Скорее всего была допущена опечатка, как в диапазоне цен, так и в средней стоимости чашки кофе.  
Заменим аналогично данное значение на NaN:

In [ ]:
data.loc[data['middle_coffee_cup'] > 1000, 'middle_coffee_cup'] = np.nan

In [ ]:
# посмотрим медианные значения и квартили столбца с помощью метода describe()
# исключим NaN и нули
cup = data[(~data['middle_coffee_cup'].isna()) & (data['middle_coffee_cup'] != 0)]

cup['middle_coffee_cup'].describe()

Аномалии обработаны, выводы сделанные из графика подтвердились. Средняя стоимость чашки кофе составляет 170 рублей.

### Работа с пропусками  

In [ ]:
# проверим, есть ли пропущенные значения в датафрейме
data.isna().sum()

In [ ]:
# посмотрим на процент пропусков относительно всех данных
pd.DataFrame(
    round(data.isna().mean()*100).sort_values(ascending=False)).style.background_gradient(cmap='coolwarm')

В данных очень большое количество пропусков, в некоторых столбцах достигает 95%.  
Несмотря на это, заполнить пропуски качественно в данном случае не получится. Средний размер счета, категория цен, количество посадочных мест и часы работы в каждом заведении индивидуальны, поэтому любое заполнение может исказить дальнейшие результаты исследования. Предлагаю не трогать пропущенные значения и двигаться дальше.

### Добавление данных 

#### Cохраним названия улиц в отдельный столбец street

In [ ]:
data['street'] = data['address'].str.split(', ').str[1]
data.head()

In [ ]:
data['street'].unique()

#### Выделим столбец is_24/7 чтобы обозначить, работает ли заведение ежедневно и круглосуточно (24/7)

In [ ]:
# укажем значение True — если заведение работает ежедневно и круглосуточно;
# False — в противоположном случае

data['is_24/7'] = data['hours'].str.contains('ежедневно') & data['hours'].str.contains('круглосуточно')
data.head()

На этом предобработка данных завершена, переходим к этапу анализа данных.

## Анализ данных

### Изучим, какие категории заведений представлены в данных:

In [ ]:
# сгруппируем заведения по категориям и посчитаем их количество в каждой из категорий
group_by_cat = data.groupby('category').agg(
    {'name': 'count'}).sort_values('name', ascending=False).reset_index()

# зададим названия столбцам
group_by_cat.columns = ['category', 'amount']
# посчитаем долю заведений в каждой из категорий по отношению к их общему количеству
group_by_cat['share_of_all'] = round(group_by_cat['amount'] / data['name'].count() * 100, 1)
group_by_cat

In [ ]:
# визуализируем распределение заведений по категориям
import warnings
warnings.simplefilter('ignore')

sns.countplot(data['category']).set(title='Распределение заведений по категориям',
                                    xlabel='Категория',
                                    ylabel='Количество заведений')
plt.xticks(rotation=45)
plt.show()

В данных представлены следующие категории заведений:  
- кафе  
- рестораны  
- кофейни   
- пиццерии  
- бары и пабы  
- булочные  
- столовые  
- заведения быстрого питания  

Большую часть заведений занимают кафе (≈ 30%) и рестораны (≈ 25%), на третьем месте находятся кофейни (≈ 17%).

### Исследуем количество посадочных мест в заведениях по категориям: 

In [ ]:
# сгруппируем заведения по категориям и посчитаем количество посадочных в каждой из категорий
seats = data.groupby('category').agg(
    {'seats': 'median'}).sort_values('seats', ascending=False).reset_index()
seats

In [ ]:
# визуализируем количество посадочных мест в заведениях по категориям
sns.barplot(data=seats, 
            x='category', 
            y='seats').set(title='Среднее количество посадочных мест по типам объектов',
                           xlabel='Категория заведения',
                           ylabel='Количество посадочных мест')
plt.xticks(rotation=45)
plt.tight_layout()

Самыми просторными заведениями являются рестораны (в среднем 86 посадочных мест), далее следуют бары и пабы (85 мест), и замыкают тройку лидеров кофейни (80 мест).

### Рассмотрим соотношение сетевых и несетевых заведений и выясним, каких заведений больше:

In [ ]:
data['chain'].value_counts()

In [ ]:
# напишем функцию для отображения значений в процентах
def autopct_format(values):
        def my_format(pct):
            total = sum(values)
            val = int(round(pct*total/100.0))
            return '{:.1f}%\n({v:d})'.format(pct, v=val)
        return my_format

# строим график
s = data['chain'].value_counts()
n = ['Не сетевые заведения', 'Сетевые заведения']
plt.pie(s,labels = n, autopct=autopct_format(s))
plt.show()

Не сетевых заведений в Москве примерно в 1,5 раза больше, чем сетевых (4538 и 2877 заведений соответственно).

### Проверим, какие категории заведений чаще являются сетевыми:

In [ ]:
# заменим 1 и 0 для удобства на ответы "да" и "нет"
data['chain'] = data['chain'].replace({1: 'да',
                                       0: 'нет'}, regex=True)

In [ ]:
# сгруппируем данные по категории и признаку сети и посчитаем количество заведений
group_by_chain = data.groupby(
    ['category', 'chain'])['name'].count().sort_values(ascending=False).reset_index()
group_by_chain.columns = ['category','chain','amount']
group_by_chain

In [ ]:
# объединим данные по общему количеству заведений в каждой категории с расчетами выше
group_by_chain.rename(columns={'amount': 'amount_chain'}, inplace=True)
total_amount = group_by_cat.merge(group_by_chain, on='category', how='left')

# посчитаем % заведений от общего количества для каждой из групп
total_amount['percent'] = round(total_amount['amount_chain'] / total_amount['amount'] * 100,1)
total_amount

In [ ]:
# визуализируем распределение
fig = px.bar(total_amount, 
             x='category', 
             y='amount_chain', 
             color='chain',
             text='percent',
             title='Количество сетевых и несетевых заведений в каждой из категорий')

fig.update_layout(xaxis_title='Категория',
                  yaxis_title='Количество заведений')

fig.show('notebook')

Самые большими категориями несетевых заведений являются бары и пабы (76.2% от всех заведений в категории), столовые (73.5%), рестораны (64.8%) и кафе (64%).   

Больше всего сетей представлено среди кофеен (48.5% от всех заведений в категории) и пиццерий (47%). 

### Сгруппируем данные по названиям заведений и найдем топ-15 популярных сетей в Москве:  

In [ ]:
top15 = data.query('chain == "да"').groupby(['name', 'category']).agg(
    {'address': 'count'}).sort_values(by='address', ascending=False).reset_index().head(15)
top15.columns = ['chain', 'category', 'amount']

top15

In [ ]:
# визуализируем топ15 сетей
fig = px.bar(top15, 
             x='chain', 
             y='amount', 
             color='category',
             title='Топ-15 популярных сетей в Москве')   

fig.update_layout(xaxis_title='Название сети',
                  yaxis_title='Количество заведений')
             
fig.show('notebook')

Самой большой и популярной сетью общественного питания в Москве является Шоколадница (открыто 120 заведений).  
В топ-15 популярных сетей представлено  6 кофеен, 4 кафе, 3 ресторана, одна пиццерия и одна булочная.   
Таким образом, можем сделать вывод о том, что самой популярной категорией для сетевых заведений являются кофейни.

### Посчитаем, сколько заведений присутствует в каждом из административных районов Москвы:

In [ ]:
group_by_district = data.groupby('district').agg(
    {'address': 'count'}).sort_values(by='address', ascending=False).reset_index()
group_by_district.columns = ['district', 'amount_of_places']
group_by_district

### Отобразим общее количество заведений и количество заведений каждой категории по районам:

In [ ]:
# сгруппируем данные по району и по категории
district_category = data.groupby(
    ['district','category'], as_index=False)['name'].count().sort_values(by='name', ascending=False)

# объединим данные по общему количеству заведений в каждом районе с расчетами выше
district_category.rename(columns={'name': 'amount_by_cat'}, inplace=True)
total_amount = group_by_district.merge(district_category, on='district', how='left')

# посчитаем % заведений от общего количества для каждой из групп
total_amount['percent'] = round(total_amount['amount_by_cat'] / total_amount['amount_of_places'] * 100,1)
total_amount.head()

In [ ]:
# построим распределение заведений по районам
fig = px.bar(total_amount.sort_values(by='amount_of_places', ascending=True), 
             x='amount_by_cat',
             y='district',               
             color='category'
            )
fig.update_layout(title='Количество заведений каждой категории по районам',
                   xaxis_title='Количество заведений',
                   yaxis_title='Район')
fig.show()

Многократно больше по отношению к остальным районам расположено заведений в Центральном административном округе.  
По другим районам заведения распределены равномерно везде, кроме Северо-Западного административного округа. В этом районе отмечается самое небольшое количество заведений.   

В разрезе категорий заведений отмечается схожая тенденция для всех районов: большая доля кафе, ресторанов и кофеен. В Центральном административном округе в отличие от других районов расположено также много баров и пабов (313 заведений).

### Посмотрим на распределение средних рейтингов по категориям заведений:

In [ ]:
mean_rating = data.groupby(
    'category', as_index=False)['rating'].mean().sort_values(by='rating', ascending=False)

mean_rating = round(mean_rating,2)
mean_rating

In [ ]:
# визуализируем распределение средних рейтингов
fig = px.line(mean_rating, 
              x='category', 
              y='rating', 
              title='',
              text="rating")

# зададим отображение значений рейтинга
fig.update_traces(textposition="bottom left")

# зададим названия осям x и y
fig.update_layout(title='Распределение средних рейтингов по категориям заведений',
                  xaxis_title='Категория заведений',
                  yaxis_title='Средний рейтинг')
fig.show() 

Усреднённые рейтинги в разных типах заведений различаются незначительно.  
Все категории имеют похожий рейтинг, от 4.04 до 4.39.

### Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района:

In [ ]:
# сгруппируем заведения по районам и посчитаем средний рейтинг
rating_df = data.groupby('district', as_index=False)['rating'].agg('mean')
rating_df

In [ ]:
# импортируем карту и хороплет
from folium import Map, Choropleth

In [ ]:
# загружаем JSON-файл с границами округов Москвы

import requests

url = 'https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson'
response = requests.get(url)
geo_json = response.json()

# задаем широту и долготу центра Москвы (moscow_lat - широта, moscow_lng - долгота)
moscow_lat, moscow_lng = 55.751244, 37.618423

In [ ]:
# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=geo_json,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.5,
    legend_name='Средний рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

### Отобразим все заведения датасета на карте  
Для этого используем кластеры из библиотеки folium:

In [ ]:
# импортируем карту и маркер
from folium import Marker
# импортируем кластер
from folium.plugins import MarkerCluster

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# выводим карту
m

### Найдем топ-15 улиц по количеству заведений:

Постройте график распределения количества заведений и их категорий по этим улицам. Попробуйте проиллюстрировать эту информацию одним графиком.

In [ ]:
# сгруппируем данные по улице и посчитаем количество заведений
group_by_street = data.groupby(
    'street', as_index=False)['name'].count().sort_values(by='name', ascending=False).head(15)

group_by_street.columns = ['street', 'amount_of_places']
group_by_street

In [ ]:
# сгруппируем данные по улице и по категории
street_category = data.groupby(
    ['street','category'], as_index=False)['name'].count().sort_values(by='name', ascending=False)

# объединим данные по общему количеству заведений в каждом районе с расчетами выше
street_category.rename(columns={'name': 'amount_by_cat'}, inplace=True)
total_amount = group_by_street.merge(street_category, on='street', how='left')

# посчитаем % заведений от общего количества для каждой из групп
total_amount['percent'] = round(total_amount['amount_by_cat'] / total_amount['amount_of_places'] * 100,1)
total_amount

In [ ]:
# посмотрим на суммарное количество заведений в каждой категории на топ-15 улицах
total_amount.groupby('category')['amount_by_cat'].sum().sort_values(ascending=False)

In [ ]:
# визуализируем распределение заведений по улицам и категориям
fig = px.bar(total_amount.sort_values(by='amount_of_places', ascending=False), 
             x='amount_by_cat',
             y='street',               
             color='category'
            )
fig.update_layout(title='Количество заведений каждой категории на топ-15 улиц',
                   xaxis_title='Количество заведений',
                   yaxis_title='Улица')
fig.show()

Больше всего заведений расположено на проспекте Мира (161 заведение), второй по популярности является Профсоюзная улица	 (106 заведений), на третьем месте находится Ленинский проспект (95 заведений).   

Больше всего на топ-15 улицах Москвы представлено кафе (319 заведений), ресторанов (281 заведение) и 205 кофеен.

### Найдем улицы, на которых находится только один объект общепита:

In [ ]:
# сгруппируем данные по улицам и районам и посчитаем количество заведений
one_object = data.groupby(['street','district'], as_index=False)['name'].count().sort_values(by='name')

# оставим только улицы с одним объектом общественного питания
one_object = one_object.query('name == 1')

# отобразим количество улиц
print('Количество улиц с одним объектом общественного питания:', one_object.shape[0]) 

In [ ]:
# посмотрим в каких районах Москвы больше всего улиц с одним заведением
dist = one_object.groupby(
    'district')['street'].count().sort_values(ascending=False).reset_index().head(10)

dist

In [ ]:
# визуализируем распределение
sns.barplot(x='street', 
            y='district', 
            data=dist).set(title='Распределение по районам количества улиц с одним заведением',
                           xlabel='Количество улиц',
                           ylabel='Районы')
            
plt.show()

Заведений, которые находятся в единственном числе на улице, на которой расположены, всего 481, это около 5% от всего количества заведений в городе. Большая часть таких заведений расположена в Центральном административном округе.

### Посчитаем медианные значения средних чеков заведений  в зависимости от района:

In [ ]:
# сгруппируем заведения по районам и посчитаем медианный чек
bill_df = data.groupby('district', as_index=False)['middle_avg_bill'].agg('median')
bill_df.sort_values('middle_avg_bill', ascending=False)

### Построим фоновую картограмму (хороплет) с полученными значениями для каждого района  
Медианный средний чек используем в качестве ценового индикатора района:

In [ ]:
# создаём карту Москвы
m2 = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создадим хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=geo_json,
    data=bill_df,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='PuBuGn',
    fill_opacity=0.5,
    legend_name='Медианный чек по районам',
).add_to(m2)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m2)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['middle_avg_bill']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# выводим карту
m2

Самые дорогие районы по среднему чеку Центральный административный округ и Западный административный округ, тут средний чек составляет 1000 рублей. Вероятнее всего это связано с тем, что в центре Москвы расположено много заведений с высоким средним чеком. На западе Москвы находится множество мест отдыха и культурного наследия, что привлекает жителей и гостей города, и как следствие на прямую влияет на то, какого ценового сегмента заведения тут будут располагаться.

В районах, расположенных в правой части города (Северо-Восточный административный округ, Восточный административный округ, Юго-Восточный административный округ и Южный административный округ), средний чек заметно ниже: от 450 до 600 рублей. 

В условной левой части города (Юго-Западный административный округе, Западный административном округе, Северо-Западном административном округе и Северном административном округе, средний чек выше и находится в диапазоне от 600 до 1000 рублей.

### Посмотрим на распределение круглосуточных заведений по категориям и районам:

In [ ]:
# сгруппируем данные по признаку круглосуточности
around_the_clock = data.groupby(
    'is_24/7', as_index=False)['name'].count().sort_values(by='name', ascending=False)
around_the_clock.columns = ['is_24/7', 'amount_of_places']
around_the_clock

In [ ]:
# сгруппируем заведения по районам и признаку круглосуточности и посчитаем их количество
opening_hours = data.groupby(
    ['district', 'is_24/7'], as_index=False)['name'].agg(
    'count').sort_values('name', ascending=False)

# объединим данные по количеству заведений по районам с расчетами выше
opening_hours.rename(columns={'name': '24_by_dist'}, inplace=True)
total_amount = group_by_district.merge(opening_hours, on='district', how='left')

# посчитаем % круглосуточных заведений от общего количества для каждого района
total_amount['percent_24'] = round(total_amount['24_by_dist'] / total_amount['amount_of_places'] * 100,1)
total_amount

In [ ]:
# визуализируем распределение круглосуточных и не круглосуточных заведений по районам
fig = px.bar(total_amount.sort_values(by='amount_of_places', ascending=True), 
             x='24_by_dist', 
             y='district', 
             color='is_24/7',
            title='Распределение круглосуточных заведений по районам')

fig.update_layout(xaxis_title='Количество заведений',
                  yaxis_title='Район')

fig.show('notebook')

## Общий вывод по исследуемым данным

В исходных данных было представлено 8406 заведений общественного питания. После обработки данных, заведений осталось 7415, и нас основе их анализа были выявлены следующие особенности рынка общественного питания Москвы:  

- В городе большую часть заведений это кафе, рестораны и кофейни. Суммарно на эти три категории приходится 70.8% всех заведений общественного питания Москвы. Причиной такого распределения могут быть универсальный формат заведений, которые подходят под любой случай и могут быть адаптированы под любые предпочтения посетителей. Остальные категории заведений в значительно меньшей степени представлены в городе и стоит предположить, что это вызвано их узкой направленностью.     


- По вместительности гостей лидирующие позиции у ресторанов (в среднем 86 посадочных мест), баров и пабов (85 мест) и кофеен (80 мест). Несмотря на то, что бары и пабы представленны в городе в меньшем количестве, они довольно объемны и готовы принять большое количество посетителей. Рестораны и кофейни рассчитаны на большой поток посетителей и не удивительно, что они располагают достаточно большой вместимостью. Самые скромные по посадочным местам булочные, среднее значение в этой категории около 50 мест.

- Большинство общественных заведений в городе относятся к несетевым — 61.2%, в то время как сетевые заведения составляют 38.8%. Бары и пабы являются самой распространенной категорией частных заведений, на частные заведение приходится 76.2% от всех баров и пабов Москвы. Булочные являются самой распространенной категорией сетевых заведений, сетевые булочные составляют значительную долю в своей категории, они занимают 64.9% от общего числа всех булочных заведений.  

- Среди топ-15 заведений в Москве примерно одна треть — кофейни. Это объясняется удобным форматом напитков и перекусов на вынос, быстрым обслуживанием, а также возможностью назначить деловую или дружескую встречу в неформальной обстановке.  

- Среди всех районов Центральный округ является наиболее оживленным и востребованным у посетителей. Центр города всегда выделялся большой проходимостью за счет притока туристов, культурной и образовательной активности, а также исторического наследия. Это подтверждается количеством открытых заведений — 1986, что составляет около 30% от общего числа заведений в городе. Северо-Западный административный округ наименее привлекателен для заведений общественного питания, в нем расположено всего 364 заведения, что составляет около 5% от общего числа заведений в городе.  

- Бары и пабы отмечены самым высоким рейтингом, средняя оценка 4.4, что является максимальным значением среди всех имеющихся категорий. Такие высокие оценки могут быть обусловлены более индивидуальным подходом к клиентам, приятной атмосферой и привлекательным интерьером. Заведения быстрого питания имеют наименьший рейтинг около 4 баллов. Это может быть объяснено тем, что такие заведения ориентированы в первую очередедь на быстрое недорогое обслуживание и большой поток посетилелей, а предоставление качественного обслуживания находится не на первом месте.  

- Самое большое количество заведений на проспект Мира — 164 заведения и на Профсоюзной улице — 106 заведений. На остальных улицах, шоссе и проспектах заведений меньше 100. Распределение категорий заведений в топ-15 улиц соответствует общему распределению по городу: больше всего кафе, ресторанов и кофеен.  

- Улиц, где есть только одно заведение, всего 481. Большая часть таких улиц расположена в Центральном административном округе.  

- Самые дорогие районы по среднему чеку Центральный административный округ и Западный административный округ - 1000 рублей, самый маленький средний чек в Южном административном округе — 450 рублей.

## Детализируем исследование: открытие кофейни

Основателям фонда «Shut Up and Take My Money» не даёт покоя успех сериала «Друзья». Их мечта — открыть такую же крутую и доступную, как «Central Perk», кофейню в Москве. Будем считать, что заказчики не боятся конкуренции в этой сфере, ведь кофеен в больших городах уже достаточно. Попробуйте определить, осуществима ли мечта клиентов.

### Посчитаем, сколько всего кофеен в датасете:

In [ ]:
# сохраним отдельно данные по кофейням
coffee_shop = data[data['category'] == 'кофейня']

print('Количество кофеен:', coffee_shop.shape[0])

### Определим, в каких районах кофеен больше всего и выявим особенности их расположения:

In [ ]:
# посчитаем количество кофеен по районам 
coffee_by_dist = coffee_shop.groupby('district', as_index=False)['name'].agg(
    'count').sort_values('name', ascending=False)
coffee_by_dist.columns = ['district', 'amount_of_places']
coffee_by_dist

In [ ]:
# визуализируем распределение кофеен по районам
sns.barplot(data=coffee_by_dist, 
            x='amount_of_places', 
            y='district').set(title='Распределение кофеен по районам',
                           xlabel='Количество заведений',
                           ylabel='Район')
plt.show()

Больше всего кофеен расположено в Центральном административном округе (381 заведение).  
В Северном, Северо-Восточном, Западном и Южном административных округах, количество кофеен находится в диапазоне от 117 до 165 шт.   
Меньше всего кофеен представлено в Восточном, Юго-Восточном, Юго-Западном и Северо-Западном административных округах, от 58 до 93 заведений.  


### Проверим, есть ли среди них круглосуточные кофейни:

In [ ]:
# посмотрим на распределение кофеен в зависимости от признака круглосуточности
s = coffee_shop['is_24/7'].value_counts()
n = ['Не круглосуточные кофейни', 'Круглосуточные кофейни']
plt.pie(s,labels = n, autopct=autopct_format(s))
plt.show()

In [ ]:
# сгруппируем кофейни по районам и признаку круглосуточности и посчитаем их количество
coffee_df = coffee_shop.groupby(
    ['district', 'is_24/7'], as_index=False)['name'].agg(
    'count').sort_values('name', ascending=False)


# визуализируем распределение круглосуточных и не круглосуточных кофеен
fig = px.bar(coffee_df, 
             x='name', 
             y='district', 
             color='is_24/7',
            title='Распределение круглосуточных кофеен по районам')

fig.update_layout(xaxis_title='Количество заведений',
                  yaxis_title='Район')

fig.show('notebook')

Круглосуточные кофейни присутствуют в каждом районе, но доля заведений, работающих 24/7, крайне маленькая.   
Больше всего таких заведений отмечено в Центральном административном округе (23 круглосуточные кофейни), в остальных районах количество круглосуточных кофеен не превышает десяти. Меньше всего круглосуточных кофеен в Южном административном округе - тут обнаружена всего одна кофейня такого типа.

### Проверим, какие у кофеен рейтинги и как они распределены по районам:

In [ ]:
# посмотрим на распределение средних рейтингов кофеен по районам
grouped_by_rating = coffee_shop.groupby(
    'district', as_index=False)['rating'].mean().sort_values(by='rating', ascending=False)
grouped_by_rating

In [ ]:
# сгруппируем кофейни по районам и рейтингу
coffee_df = coffee_shop.groupby(
    ['district', 'rating'], as_index=False)['name'].agg(
    'count').sort_values('name', ascending=False)

# объединим данные по общему рейтингу кофеен с расчетами выше
coffee_df.rename(columns={'name': 'amount_by_dist'}, inplace=True)
total_amount = coffee_by_dist.merge(coffee_df, on='district', how='left')

# визуализируем распределение рейтинга кофеен по районам
fig = px.bar(total_amount.sort_values(by='amount_of_places', ascending=True), 
             x='amount_by_dist', 
             y='district', 
             color='rating',
            title='Распределение рейтинга кофеен по районам')

fig.update_layout(xaxis_title='Рейтинг',
                  yaxis_title='Район')

fig.show('notebook')

Разброс рейтингов кофеен по городу не очень велик, многие кофейни имеют рейтинг от 4 баллов и выше.   
Большинство заведений с высоким рейтингом (от 4 баллов) расположены в Центральном и Северо-Западном административных округах. Кофейни с рейтингом ниже 4 баллов преимущественно располагаются в Юго-Восточном, Южном, Северо-Восточном и Западном административных округах. 

### Определим, на какую стоимость чашки капучино стоит ориентироваться при открытии и почему:

Определим медианное значение цен на чашку капучино по каждому округу Москвы:

In [ ]:
price_coffeeshop = coffee_shop.groupby(
    'district', as_index=False)['middle_coffee_cup'].median().sort_values(
     by='middle_coffee_cup', ascending=False)

price_coffeeshop

In [ ]:
# визуализируем распределение медианной стоимости чашки капучино по районам
fig = px.bar(price_coffeeshop.sort_values(by='middle_coffee_cup', ascending=False), 
             x='middle_coffee_cup', 
             y='district',    
             color='district',
             text='middle_coffee_cup', 
             title='Медианная цена на чашку кофе')
            
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(yaxis_title='Округ',
                  xaxis_title='Средняя цена',
                  xaxis=dict(range=[120, 210]),
                  showlegend=False, legend_title='Округ')
fig.show() 

Имеет смысл смотреть на ценовую политику кофейни по медианному значению чашки капучино, как ключевого напитка, который есть в каждой кофейне. Распределение цен в кофейнях по районам остается практически таким же, как и при сравнении медианных цен по всем заведениям. Самыми дорогими округами являются Центральный и Западный административные округа — 185 рублей за чашку, недалеко от них расположился Юго-Западный административный округ - 182 рубля за чашку. Самое дешевое капучино в Восточном административном округе — 135 рублей.

## Рекомендации по открытию кофейни

Центральный административный округ является наиболее популярным районом для открытия кофеен в Москве, с наибольшим количеством заведений — 381 кофейня и высоким рейтингом — 4.3 балла. Северный административный округ также представлен большим количеством кофеен — 165, что указывает с одной стороны на большой спрос со стороны покупателей, а с другой на высокую конкуренцию в этих районах. 

Северо-Западный административный округ также имеет высокий рейтинг кофеен — 4.3, при этом количество заведений здесь гораздо меньше - открыто всего 58 кофеен, что может послужить благоприятными условиями для открытия нового заведения. Небольшое количество кофеен в Северо-Западном административном округе может быть связано с малой плотностью населения и преобладанием водных ресурсов и лесных массивов. Однако, это может представлять потенциал для развития новых кофейных заведений с уникальными концепциями, которые органично будут встроены в атмосферу округа. 

Большинство кофеен (95%) работают только в дневное время, что говорит о большом потенциале для открытия для открытия круглосуточной кофейни, особенно в центре города.

Цены на кофе варьируются в зависимости от округа. Центральный и Западный административные округа имеют самые высокие цены на капучино (185 рублей за чашку), в то время как Восточный административный округ имеет самые низкие цены (135 рублей за чашку). При выборе места для открытия кофейни, следует учитывать предпочтения и потребности целевой аудитории в соотношении цены и качества кофе.

Презентация: <https://disk.yandex.ru/d/I5xdivQTuvEcHA>  